In [1]:
# Import Libraries
import pandas as pd
import requests
import time

In [34]:
API_KEY = 'xxxxx'
CHANNEL_ID = 'UCGX7nGXpz-CmO_Arg-cgJ7A'

In [36]:
def get_video_details(video_id):

    #collecting view, like, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, comment_count

In [38]:

def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title=str(video_title).replace("#","")
                video_title=str(video_title).replace(";","e ")
                video_title=str(video_title).replace("39e","")
                video_title=str(video_title).replace("quote","")
                video_title=str(video_title).replace("&","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [40]:
#main

#build our dataframe
df2 = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"]) 

df2 = get_videos(df2)
     

/var/folders/k_/qfxxg3lx25d_vts4gmjyqvy40000gn/T/ipykernel_45019/3671529603.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'video_id':video_id,'video_title':video_title,
/var/folders/k_/qfxxg3lx25d_vts4gmjyqvy40000gn/T/ipykernel_45019/3671529603.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'video_id':video_id,'video_title':video_title,
/var/folders/k_/qfxxg3lx25d_vts4gmjyqvy40000gn/T/ipykernel_45019/3671529603.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'video_id':video_id,'video_title':video_title,
/var/folders/k_/qfxxg3lx25d_vts4gmjyqvy40000gn/T/ipykernel_45019/3671529603.py:21: FutureWarning: The frame.append method is deprecated and will be removed f

In [42]:

df2

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,ZlzNpZ7kfXg,I d Be Mad If I Didn t Go To Prison - Tristo...,2023-06-17,51456,3783,94
1,HyTn4m_iysQ,They Took Everything That Was Valuable - Tri...,2023-06-17,238610,14147,278
2,aJf03-m1d6o,There s 3 Different Types of People in The World,2023-06-16,41408,2552,82
3,k5nUDSpdIfA,She Said Nothing Illegal Happened - Tristan ...,2023-06-16,383537,18390,220
4,TxLJEzG1T38,Tristan Tate EXCLUSIVE INTERVIEW - Jail | Brot...,2023-06-16,3699633,157752,11023
...,...,...,...,...,...,...
479,khZpELHbDKs,Bet-David Podcast | EP 40,2021-02-16,59201,2653,466
480,cBv3-fLvXZM,Election Night Marathon on the Bet-David Podca...,2020-11-04,35311,1070,359
481,vD9b66_Cz4A,Bet-David Podcast | EP 17,2020-10-06,50744,3044,358
482,ignaDGEhXUE,Bet-David Podcast | Guest: Danielle DiMartino ...,2020-10-01,55592,2956,432


In [54]:

df2.to_csv('Valuetainment_Youtube_Data.csv')